# 1.1.2.1: Definición, Representación e Independencia Lineal de Matrices

## Objetivos de Aprendizaje

Al completar este notebook, serás capaz de:

- **Definir** qué es una matriz, sus dimensiones ($m \times n$), filas y columnas, usando datasets como ejemplo.
- **Representar** matrices en Python con NumPy y acceder a sus elementos, filas y columnas.
- **Generar y reconocer** matrices especiales (identidad, diagonal, simétrica, singular) y calcular la **transpuesta**.
- **Definir** la **Independencia Lineal** de los vectores columna de una matriz y calcular su **rango**.
- **Conectar** la dependencia lineal con el problema práctico de la **multicolinealidad** en ciencia de datos.

In [ ]:
# --- Celda de Configuración (Oculta) ---
%display latex
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns # Para visualizaciones más atractivas

def plot_matrix(matrix, title="Visualización de Matriz", ax=None, cmap='viridis'):
    """Función mejorada para visualizar una matriz con un heatmap."""
    if not isinstance(matrix, np.ndarray):
        matrix = np.array(matrix, dtype=float)
    
    standalone = ax is None
    if standalone:
        fig, ax = plt.subplots(figsize=(5, 4))
    
    sns.heatmap(matrix, annot=matrix.size <= 100, fmt=".2f", cmap=cmap, cbar=True, ax=ax, linewidths=.5)
    ax.set_title(title)
    ax.tick_params(axis='both', which='both', length=0) # Ocultar ticks
    
    if standalone:
        plt.show()

--- 
## ⚙️ El Arsenal de Datasets: Nuestra Fuente de Ejercicios

Para este notebook, donde la estructura de los datos es el concepto central, nuestro arsenal de generadores es más importante que nunca. Nos permitirá crear al instante matrices con propiedades específicas para explorar cada idea.

In [ ]:
# === CONFIGURACIÓN DE DATASETS ===
from src.data_generation.create_student_performance import create_student_performance_data
from src.data_generation.create_business_data import create_business_data
from src.data_generation.create_special_matrices import create_special_matrices
from src.data_generation.create_edge_cases import create_edge_cases

# Configuración centralizada de aleatoriedad para REPRODUCIBILIDAD
rng = np.random.default_rng(seed=42)

# === Generación de Datasets para este Notebook ===

# 💡 CONTEXTO PEDAGÓGICO: Hilo Conductor (Observaciones vs Features)
# La matriz de estudiantes es el ejemplo perfecto de una matriz de datos (data matrix), 
# donde las filas son observaciones y las columnas son features.
datos_estudiantes = create_student_performance_data(rng, n_samples=50)

# 💡 CONTEXTO PEDAGÓGICO: Multicolinealidad en el Mundo Real
# Este generador crea un dataset donde una columna es una combinación lineal de las otras.
# Es el ejemplo perfecto para ilustrar la dependencia lineal y el concepto de rango.
datos_multicolineales = create_edge_cases(rng, case_type='multicollinear', n_samples=100)

# 💡 CONTEXTO PEDAGÓGICO: Matrices con Propiedades Controladas
# Para estudiar matrices especiales, nada mejor que generarlas directamente. Crearemos
# matrices singulares (columnas dependientes) y ortogonales (columnas perfectamente independientes).
matriz_singular = create_special_matrices(rng, matrix_type='singular', size=(4, 4))
matriz_aleatoria_cuadrada = create_special_matrices(rng, matrix_type='random', size=(4, 4))

print("Datasets y matrices generados y listos para usar.")
datos_estudiantes.head()

## 1. ¿Qué es una Matriz?

Si un vector es una lista de números, una **matriz** es una parrilla o tabla rectangular de números, organizada en filas y columnas. Es la estructura de datos más fundamental en álgebra lineal y ciencia de datos.

> **Conexión Directa:** Un DataFrame de Pandas o una hoja de cálculo **ES** una matriz. 
> - Las **filas** ($m$) representan las **observaciones** (e.g., 50 estudiantes).
> - Las **columnas** ($n$) representan las **features** (e.g., 'horas_estudio', 'calificacion_examen', etc.).

#### Anatomía de una Matriz
- **Dimensión (o Forma):** Se describe como **$m \times n$**, donde **$m$** es el número de filas y **$n$** es el número de columnas.
- **Elementos:** Cada número individual. Se accede a ellos mediante su posición `A[i, j]` (fila *i*, columna *j*), **recordando que los índices en Python/NumPy empiezan en 0**.

### Ejemplo Demostrativo 1: El "Hilo Conductor" como Matriz
Vamos a tomar una subsección de nuestro DataFrame de estudiantes y a tratarla explícitamente como una matriz de NumPy.

In [ ]:
# 1. Extraemos las primeras 5 filas y 3 columnas como una matriz de NumPy
matriz_muestra_estudiantes = datos_estudiantes[['horas_estudio', 'calificacion_previa', 'calificacion_examen']].head(5).values

# 2. Verificamos sus propiedades
print(f"Tipo de dato: {type(matriz_muestra_estudiantes)}")
print(f"Dimensiones (forma): {matriz_muestra_estudiantes.shape}")
print(f"Número de filas (m): {matriz_muestra_estudiantes.shape[0]}")
print(f"Número de columnas (n): {matriz_muestra_estudiantes.shape[1]}")

# 3. Accedemos a un elemento: la calificación previa (columna 1) del estudiante en la fila 3 (índice 2)
elemento_2_1 = matriz_muestra_estudiantes[2, 1]
print(f"\nElemento en la fila 3, columna 2 (índices 2,1): {elemento_2_1:.2f}")

# 4. Visualización
plot_matrix(matriz_muestra_estudiantes, title='Matriz de 5 Estudiantes x 3 Features')

---
## 2. Tipos Especiales de Matrices y la Transpuesta

Existen varias matrices con nombres especiales que aparecen constantemente:

- **Matriz Cuadrada:** Tiene el mismo número de filas y columnas ($m = n$).
- **Matriz Identidad ($I_n$):** Matriz cuadrada con 1s en la diagonal principal y 0s en el resto. Es el análogo al número 1 en la multiplicación de matrices.
- **Matriz Diagonal:** Matriz cuadrada donde solo los elementos de la diagonal principal pueden ser no nulos.
- **Matriz Simétrica:** Matriz cuadrada que es igual a su transpuesta ($A = A^T$). La matriz de covarianza es un ejemplo famoso.
- **Transpuesta ($A^T$):** La operación de "voltear" la matriz sobre su diagonal. Las filas de A se convierten en las columnas de $A^T$. Si A es $m \times n$, $A^T$ es **$n \times m$**.

### Ejemplo Demostrativo 2: Creando Matrices Especiales y la Transpuesta

In [ ]:
# 1. GENERACIÓN DE DATOS
# Usamos nuestro generador para crear una matriz simétrica.
matriz_simetrica = create_special_matrices(rng, 'symmetric', size=(4,4))

# 2. APLICACIÓN DEL CONCEPTO
# Creamos una matriz no cuadrada y una identidad
A = np.array([[1, 2, 3], [4, 5, 6]])
A_t = A.T
I = np.identity(4)

# 3. INTERPRETACIÓN Y VISUALIZACIÓN
print(f"Matriz A (forma {A.shape}):\n{A}")
print(f"\nTranspuesta A.T (forma {A_t.shape}):\n{A_t}")

fig, axs = plt.subplots(1, 3, figsize=(15, 4))
plot_matrix(I, 'Matriz Identidad 4x4', ax=axs[0], cmap='gray_r')
plot_matrix(matriz_simetrica, 'Matriz Simétrica 4x4', ax=axs[1], cmap='cividis')
plot_matrix(matriz_simetrica.T, 'Transpuesta de la Simétrica', ax=axs[2], cmap='cividis')
plt.tight_layout()
plt.show()

print("Observa cómo la matriz simétrica y su transpuesta son visualmente idénticas.")

---
## 3. Independencia Lineal: ¿Aportan mis datos información nueva?

Ahora que vemos las matrices como colecciones de vectores columna (features), podemos hacernos una pregunta crucial: **¿son todas nuestras features realmente necesarias?**

Un conjunto de vectores es **linealmente independiente** si ningún vector en el conjunto puede ser escrito como una combinación lineal de los otros. Intuitivamente, significa que cada vector (cada feature) aporta "información direccional nueva" y no es redundante.

Si los vectores columna de una matriz son **linealmente dependientes**, tenemos **multicolinealidad**. Esto significa que al menos una de nuestras features es una mezcla de las otras (e.g., `gasto_en_usd = gasto_en_eur * 1.07`). La información es redundante y puede causar problemas serios de inestabilidad en muchos modelos de Machine Learning (como la Regresión Lineal).

### La Prueba Práctica: El Rango de una Matriz

El **rango** de una matriz es el número de columnas (o filas) linealmente independientes que tiene. Es la verdadera "dimensión" de la información contenida en la matriz.

- Si `rango(A) == número de columnas`, las columnas son **linealmente independientes**.
- Si `rango(A) < número de columnas`, las columnas son **linealmente dependientes** (hay multicolinealidad).

### Ejemplo Demostrativo 3: Detectando Multicolinealidad con el Rango

In [ ]:
# 1. GENERACIÓN DE DATOS
# Usamos nuestro generador de casos de borde para crear un dataset con multicolinealidad perfecta.
# Por construcción, la columna 'x3' es una combinación de 'x1' y 'x2'.
X_dependiente = datos_multicolineales[['x1', 'x2', 'x3']].values

# Para comparar, creamos una matriz con columnas independientes
X_independiente = rng.random(size=(X_dependiente.shape[0], 3))

# 2. APLICACIÓN DEL CONCEPTO: Calcular el rango
rango_dependiente = np.linalg.matrix_rank(X_dependiente)
rango_independiente = np.linalg.matrix_rank(X_independiente)

# 3. INTERPRETACIÓN
print("--- Caso 1: Matriz con Multicolinealidad ---")
print(f"Forma de la matriz: {X_dependiente.shape}")
print(f"Rango de la matriz: {rango_dependiente}")
if rango_dependiente < X_dependiente.shape[1]:
    print("🔴 ¡Alerta! El rango es menor que el número de columnas -> Hay multicolinealidad.")

print("\n--- Caso 2: Matriz Aleatoria (sin multicolinealidad esperada) ---")
print(f"Forma de la matriz: {X_independiente.shape}")
print(f"Rango de la matriz: {rango_independiente}")
if rango_independiente == X_independiente.shape[1]:
    print("🟢 El rango es igual al número de columnas -> Las features son linealmente independientes.")

### Ejemplo Demostrativo 4: Visualizando Columnas Dependientes e Independientes

In [ ]:
# 1. DATOS
# Columnas independientes (no están en la misma línea/plano)
v1 = np.array([1, 2, 3])
v2 = np.array([4, 1, 5])
v3 = np.array([-1, 3, -2])
M_indep = np.column_stack([v1, v2, v3])

# Columnas dependientes (v6 = v4 + v5)
v4 = np.array([1, 1, 1])
v5 = np.array([2, 0, 2])
v6 = v4 + v5
M_dep = np.column_stack([v4, v5, v6])

# 2. VISUALIZACIÓN
fig = plt.figure(figsize=(12, 6))
ax1 = fig.add_subplot(121, projection='3d')
ax2 = fig.add_subplot(122, projection='3d')

# Plot independiente
for i, v in enumerate([v1, v2, v3]):
    ax1.quiver(0, 0, 0, v[0], v[1], v[2], label=f'v{i+1}')
ax1.set_title(f'Columnas Independientes (Rango={np.linalg.matrix_rank(M_indep)})')
ax1.legend()

# Plot dependiente
for i, v in enumerate([v4, v5, v6]):
    ax2.quiver(0, 0, 0, v[0], v[1], v[2], label=f'v{i+4}')
ax2.set_title(f'Columnas Dependientes (Rango={np.linalg.matrix_rank(M_dep)})')
ax2.legend()

plt.show()

print("Observa cómo en el gráfico de la derecha, los 3 vectores yacen en el mismo plano. v6 no añade una nueva dirección; es redundante.")

---
## 4. Ejercicios Guiados con Scaffolding (8+)
Rellena las partes marcadas con `# COMPLETAR` para afianzar tu comprensión.

### === EJERCICIO GUIADO 1: Creación y Propiedades Básicas ===

In [ ]:
# TODO 1: Crea una matriz 2x4 en NumPy con los números del 1 al 8.
A = # COMPLETAR

# VERIFICACIÓN AUTOMÁTICA
assert isinstance(A, np.ndarray), "A no es un array de NumPy"
assert A.shape == (2, 4), f"La forma de A es {A.shape}, pero se esperaba (2, 4)"
print("✅ ¡Matriz creada correctamente!")
print(f"Matriz A:\n{A}")

### === EJERCICIO GUIADO 2: Slicing (Extracción de Sub-matrices) ===

In [ ]:
# DATOS: Una matriz 5x5 de nuestro dataset de estudiantes.
matriz_est = datos_estudiantes.head(5).values

# TODO 1: Extrae la primera fila de la matriz.
# PISTA: El slicing es A[fila, columna]. Usa ':' para 'todas'.
primera_fila = # COMPLETAR

# TODO 2: Extrae la tercera columna de la matriz.
tercera_columna = # COMPLETAR

# VERIFICACIÓN AUTOMÁTICA
assert primera_fila.shape == (5,), f"La forma de la primera fila es {primera_fila.shape}, se esperaba (5,)"
assert tercera_columna.shape == (5,), f"La forma de la tercera columna es {tercera_columna.shape}, se esperaba (5,)"
print(f"✅ ¡Slicing correcto!")
print(f"Primera Fila (Estudiante 0): {np.round(primera_fila, 2)}")
print(f"Tercera Columna (Asistencia %): {np.round(tercera_columna, 2)}")

### === EJERCICIO GUIADO 3: Transpuesta de una Matriz de Datos ===

In [ ]:
# DATOS: Una submatriz de 10x3 del dataset de negocio.
matriz_negocio = create_business_data(rng, n_samples=10)[['precio', 'gasto_marketing', 'ventas_mensuales']].values

# TODO 1: Calcula la transpuesta de 'matriz_negocio'.
matriz_transpuesta = # COMPLETAR

# VERIFICACIÓN AUTOMÁTICA
expected_shape = (matriz_negocio.shape[1], matriz_negocio.shape[0])
assert matriz_transpuesta.shape == expected_shape, f"La forma de la transpuesta es {matriz_transpuesta.shape}, se esperaba {expected_shape}"
print(f"✅ ¡Transpuesta correcta!")
print(f"Forma Original (Observaciones x Features): {matriz_negocio.shape}")
print(f"Forma Transpuesta (Features x Observaciones): {matriz_transpuesta.shape}")
print("Interpretación: Hemos cambiado de una vista de '10 productos' a una vista de '3 series de datos de longitud 10'.")

### === EJERCICIO GUIADO 4: Verificación de Matriz Simétrica ===

In [ ]:
# DATOS: Una matriz simétrica y una no simétrica.
M_simetrica = create_special_matrices(rng, 'symmetric', (3, 3))
M_no_simetrica = create_special_matrices(rng, 'random', (3, 3))

# TODO 1: Comprueba si M_simetrica es igual a su transpuesta.
# PISTA: Usa np.allclose() para comparar arrays de punto flotante de forma segura.
es_simetrica = # COMPLETAR

# TODO 2: Comprueba si M_no_simetrica es igual a su transpuesta.
no_es_simetrica = # COMPLETAR

# VERIFICACIÓN AUTOMÁTICA
assert es_simetrica == True, "La matriz simétrica debería ser igual a su transpuesta."
assert no_es_simetrica == False, "La matriz aleatoria no debería ser simétrica."
print("✅ ¡Verificación correcta!")

### === EJERCICIO GUIADO 5: Rango de una Matriz Singular ===

In [ ]:
# DATOS: Usamos la matriz singular 4x4 que generamos al principio.
# Por construcción, sus columnas son linealmente dependientes.

# TODO 1: Calcula el rango de 'matriz_singular'.
rango = # COMPLETAR

# TODO 2: Compara el rango con el número de columnas.
num_columnas = matriz_singular.shape[1]
es_dependiente = # COMPLETAR (debe ser una expresión booleana: True o False)

# VERIFICACIÓN AUTOMÁTICA
assert es_dependiente == True, "Una matriz singular debe tener columnas dependientes (rango < n)."
print("✅ ¡Análisis correcto!")
print(f"La matriz 4x4 tiene rango {rango}, confirmando que es singular y sus columnas son dependientes.")

### === EJERCICIO GUIADO 6: Rango de una Matriz de Rango Completo ===

In [ ]:
# DATOS: Usamos la matriz aleatoria cuadrada 4x4.
# Una matriz aleatoria casi con seguridad tendrá columnas independientes.

# TODO 1: Calcula el rango de 'matriz_aleatoria_cuadrada'.
rango = # COMPLETAR

# TODO 2: Compara el rango con el número de columnas.
num_columnas = matriz_aleatoria_cuadrada.shape[1]
es_independiente = # COMPLETAR (debe ser una expresión booleana)

# VERIFICACIÓN AUTOMÁTICA
assert es_independiente == True, "Una matriz aleatoria cuadrada típicamente tiene rango completo."
print("✅ ¡Análisis correcto!")
print(f"La matriz 4x4 tiene rango {rango}, confirmando que tiene rango completo y sus columnas son independientes.")

### === EJERCICIO GUIADO 7: Introduciendo Multicolinealidad Manualmente ===

In [ ]:
# DATOS: El dataset de estudiantes (solo 2 columnas).
matriz_original = datos_estudiantes[['horas_estudio', 'calificacion_previa']].values

# TODO 1: Crea una nueva columna que sea una combinación lineal de las existentes.
# Por ejemplo, nueva_col = 2 * col_0 + 3 * col_1
nueva_columna_redundante = # COMPLETAR

# TODO 2: Apila la nueva columna a la matriz original para crear una matriz con dependencia lineal.
# PISTA: Usa np.column_stack([matriz_original, nueva_columna_redundante])
matriz_dependiente = # COMPLETAR

# VERIFICACIÓN AUTOMÁTICA
rango_original = np.linalg.matrix_rank(matriz_original)
rango_dependiente = np.linalg.matrix_rank(matriz_dependiente)
assert rango_original == 2, "La matriz original debería tener rango 2."
assert rango_dependiente == 2, "Al añadir una columna dependiente, el rango NO debe aumentar."
print("✅ ¡Multicolinealidad introducida y detectada con éxito!")
print(f"Rango de la matriz original (2 columnas): {rango_original}")
print(f"Rango de la nueva matriz (3 columnas): {rango_dependiente}")

### === EJERCICIO GUIADO 8: Creando una Matriz Identidad ===

In [ ]:
# TODO 1: Crea una matriz identidad de tamaño 5x5.
# PISTA: Usa np.identity() o np.eye().
I5 = # COMPLETAR

# VERIFICACIÓN AUTOMÁTICA
assert I5.shape == (5, 5), "La forma no es 5x5."
assert np.sum(np.diag(I5)) == 5, "La suma de la diagonal de I5 debe ser 5."
assert np.sum(I5) - np.sum(np.diag(I5)) == 0, "Los elementos fuera de la diagonal deben ser 0."
print("✅ ¡Matriz identidad creada correctamente!")
plot_matrix(I5, title='Matriz Identidad 5x5', cmap='gray_r')

--- 
# 5. Banco de Ejercicios Prácticos (30+)
Ahora te toca a ti. Resuelve estos ejercicios para consolidar tu conocimiento.

### Parte A: Creación, Dimensiones y Transpuesta

**A1 (🟢 Fácil):** Crea una matriz 3x2 en NumPy con los números del 1 al 6. Imprime la matriz y su forma (`.shape`).

**A2 (🟢 Fácil):** Calcula y muestra la transpuesta de la matriz del ejercicio anterior. Verifica que su nueva forma sea 2x3.

**A3 (🟢 Fácil):** Toma la matriz `datos_estudiantes` y extrae las columnas `['tutor_privado', 'calificacion_examen']` para los primeros 10 estudiantes. Muestra la forma de esta nueva matriz.

**A4 (🟢 Fácil):** De la matriz anterior, extrae y muestra la calificación del quinto estudiante (índice 4).

**A5 (🟡 Medio):** Crea una matriz de 10x2 con datos del dataset de negocio: 'precio' y 'ventas_mensuales'. Visualízala usando `plot_matrix`.

**A6 (🟡 Medio):** Transpón la matriz del ejercicio A5. ¿Qué representan ahora las filas y las columnas? ¿Cuál es la nueva dimensión?

**A7 (🔴 Reto):** Crea una matriz de 5x5 donde cada elemento `A[i, j]` sea igual a `i + j`. Luego, calcula su transpuesta. ¿Es la matriz original simétrica?

### Parte B: Matrices Especiales

**B1 (🟢 Fácil):** Crea y visualiza una matriz identidad 6x6.

**B2 (🟢 Fácil):** Crea una matriz 4x4 con ceros en todas partes excepto en la diagonal, que debe contener los valores `[5, 1, 8, 3]`.

**B3 (🟡 Medio):** Genera una matriz simétrica de 5x5 usando `create_special_matrices`. Verifica que `A[1, 3]` es igual a `A[3, 1]`.

**B4 (🟡 Medio):** Genera una matriz ortogonal de 4x4. Aunque no hemos visto la definición completa, una propiedad clave es que $A^T A = I$. Verifica si esto se cumple para tu matriz (usa `np.allclose` para la comparación).

**B5 (🔴 Reto):** Una matriz antisimétrica es una matriz cuadrada donde $A^T = -A$. Crea una matriz aleatoria 3x3 `R`, y luego construye una matriz antisimétrica `A` calculando `A = R - R.T`. Verifica que la propiedad `A.T == -A` se cumple.

### Parte C: Independencia Lineal y Rango

**C1 (🟢 Fácil):** ¿Son los vectores columna de la matriz `A = np.array([[1, 2], [0, 1]])` linealmente independientes? Verifica calculando el rango.

**C2 (🟢 Fácil):** ¿Y los de la matriz `B = np.array([[1, 2], [2, 4]])`? Verifica con el rango y explica el resultado.

**C3 (🟡 Medio):** Determina si el conjunto de vectores `{[1, 2, 3], [4, 5, 6], [7, 8, 9]}` es linealmente independiente. (Pista: ponlos como columnas de una matriz y calcula su rango).

**C4 (🟡 Medio):** Toma las 4 primeras columnas de `datos_estudiantes`. ¿Son linealmente independientes? Calcula el rango para responder.

**C5 (🟡 Medio):** Genera una matriz singular 5x5. ¿Cuál esperas que sea su rango? Verifícalo.

**C6 (🔴 Reto):** Crea una matriz 4x4 donde la cuarta columna sea la suma de las dos primeras. Sin calcular explícitamente el rango, ¿cuál es el rango máximo posible que podría tener esta matriz? Luego, calcula el rango para verificar tu hipótesis.

**C7 (🔴 Reto):** Usa `datos_multicolineales`. Sabemos que 'x3' depende de 'x1' y 'x2'. ¿Son 'x1' y 'x2' linealmente independientes entre sí? Crea una submatriz solo con estas dos columnas y calcula su rango para confirmarlo.

---

## ✅ Mini-Quiz de Autoevaluación

1. Si una matriz de datos de 100 observaciones y 5 features tiene un rango de 4, ¿qué significa esto sobre las features?
2. ¿Cuál es la dimensión de la transpuesta de una matriz 7x3?
3. Verdadero o Falso: El elemento en `A[1, 2]` de una matriz NumPy es el que está en la primera fila y la segunda columna.
4. ¿Qué propiedad define a una matriz simétrica en relación con su transpuesta?
5. ¿Cuál es el rango de una matriz identidad 10x10?

## 🚀 Próximos Pasos

¡Felicidades! Ahora entiendes la estructura fundamental para organizar datos y cómo analizar su calidad con el concepto de independencia lineal.

- En el notebook **`1.1.2.2_Operaciones_Matriciales.ipynb`**, aprenderás a sumar, restar y multiplicar matrices. Estas operaciones son el motor de casi todos los algoritmos de Machine Learning, desde la regresión lineal hasta las redes neuronales.